In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import precision_score, recall_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [2]:
train_df = pd.read_csv('../input/Gtrain.csv');
test_df = pd.read_csv('../input/Gtest.csv')
commondata = [train_df, test_df] 
origtraindata = train_df.copy()
origtestdata = test_df.copy()
ID  = test_df['ID']
le = preprocessing.LabelEncoder()

In [3]:
train_df.head()

,ID,Date,Opponent,Is_Home_or_Away,Is_Opponent_in_AP25_Preseason,Media,Label
0,1,9/5/2015,Texas,Home,Out,1-NBC,Win
1,2,9/12/2015,Virginia,Away,Out,4-ABC,Win
2,3,9/19/2015,GeorgiaTech,Home,In,1-NBC,Win
3,4,9/26/2015,UMass,Home,Out,1-NBC,Win
4,5,10/3/2015,Clemson,Away,In,4-ABC,Lose


In [4]:
test_df.head()

,ID,Date,Opponent,Is_Home_or_Away,Is_Opponent_in_AP25_Preseason,Media,Label
0,25,9/2/2017,Temple,Home,Out,1-NBC,Win
1,26,9/9/2017,Georgia,Home,In,1-NBC,Lose
2,27,9/16/2017,BostonCollege,Away,Out,2-ESPN,Win
3,28,9/23/2017,MichiganState,Away,Out,3-FOX,Win
4,29,9/30/2017,MiamiOhio,Home,Out,1-NBC,Win


In [5]:
# data preprocessing encoding values
for data in commondata:
    data['Media'] = le.fit_transform(data['Media'])
    data['Label'] = le.fit_transform(data['Label'])
    data['Is_Home_or_Away'] =  le.fit_transform(data['Is_Home_or_Away'])
    data['Is_Opponent_in_AP25_Preseason'] =  le.fit_transform(data['Is_Opponent_in_AP25_Preseason'])
    data['Opponent'] =  le.fit_transform(data['Opponent'])


In [6]:
train_df = train_df.drop('Date', axis='columns')
test_df = test_df.drop('Date', axis='columns')

In [7]:
train_df.head()

,ID,Opponent,Is_Home_or_Away,Is_Opponent_in_AP25_Preseason,Media,Label
0,1,14,1,1,0,1
1,2,17,0,1,3,1
2,3,4,1,0,0,1
3,4,15,1,1,0,1
4,5,2,0,0,3,0


In [8]:
test_df.head()

,ID,Opponent,Is_Home_or_Away,Is_Opponent_in_AP25_Preseason,Media,Label
0,25,9,1,1,0,1
1,26,1,1,0,0,0
2,27,0,0,1,1,1
3,28,4,0,1,2,1
4,29,3,1,1,0,1


In [9]:
# Naive Bayesian Classification
X = train_df.drop('Label', axis='columns')
Y = train_df['Label']
ytrue = test_df['Label']
test_df = test_df.drop('Label', axis='columns')
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.5, random_state=0)

gnb = GaussianNB()
ypred = gnb.fit(train_X,train_Y).predict(test_X)

In [10]:
print("Naive Bayes model Accuracy:", accuracy_score (test_Y, ypred))

Naive Bayes model Accuracy: 0.5833333333333334


In [11]:
ypredt = gnb.predict(test_df)
predictions = pd.DataFrame(
    data={'ID': ID,
          'Label': ypredt})
predictions.to_csv('FG_Naive_predictions.csv', index=False)

In [12]:
prec = precision_score(ytrue, ypredt)
recall = recall_score(ytrue, ypredt)
nume = 2* prec * recall
den = prec+recall
F1Score = nume/den
print("Precision of Naive Bayes is :", prec)
print("Recall of Naive Bayes is : ", recall)
print("F1Score of Naive Bayes is : ", F1Score)

Precision of Naive Bayes is : 1.0
Recall of Naive Bayes is :  0.6666666666666666
F1Score of Naive Bayes is :  0.8


In [13]:
print("Accuracy of Naive Bayes is :", accuracy_score (ytrue, ypredt))

Accuracy of Naive Bayes is : 0.75


In [14]:
#KNN
knn = KNeighborsClassifier(n_neighbors=3).fit(X, Y)
ypred_KNN = knn.predict(test_df)

In [15]:
KNN_pred = pd.DataFrame(
    data={'ID': ID,
          'Label': ypred_KNN})

In [16]:
KNN_pred.to_csv('FG_KNN_predictions.csv', index=False)

In [17]:
KNN_prec = precision_score(ytrue, ypred_KNN)
KNN_recall = recall_score(ytrue, ypred_KNN )
n = 2* KNN_prec * KNN_recall
d = KNN_prec+ KNN_recall
KNN_F1Score = n/d
print("Precision of KNN is : ", KNN_prec)
print("Recall of KNN is : ", KNN_recall)
print("F1Score of KNN is : ", KNN_F1Score)

Precision of KNN is :  0.75
Recall of KNN is :  0.6666666666666666
F1Score of KNN is :  0.7058823529411765


In [18]:
print("Accuracy of KNN is :", accuracy_score (ytrue, ypred_KNN))

Accuracy of KNN is : 0.5833333333333334


In [19]:
ypred_KNN

array([0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0])

In [20]:
ypredt

array([1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0])

In [21]:
ytrue

0     1
1     0
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     0
10    1
11    0
Name: Label, dtype: int32